# **Voice of Customer**

Ampika Thanakaew ID: 6210422049

## **Initialize**

In [1]:
# !pip install mlxtend
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [4]:
import pandas as pd
import numpy as np
import re
import gensim
import pythainlp
from pythainlp.tokenize import sent_tokenize, word_tokenize
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [6]:
%cd '/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer/'
%ls

/content/drive/MyDrive/BADS 7105 CRM/Voice of Customer
CustomerReviews.csv


In [7]:
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('/content/drive/MyDrive/BADS 7105 CRM/font/Cordia New Bold.ttf')
mpl.rc('font', family='Cordia New', size=12, weight=200)

**Load Data**

In [30]:
df = pd.read_csv('CustomerReviews.csv')
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


**Tokenize Words**

In [31]:
# Word Correction
word_correction_dict = {
    'โมโม่' : 'momo', 'มากก' : 'มาก', 'กกก' : 'ก', 'แวท' : 'vat', 'พิซเซอเรีย' : 'pizzeria', 'พิซซ่า' : 'pizza', 
    'เก้บ' : 'เก็บ', 'ชูชิ' : 'ซูชิ', 'ซึป' : 'ซุป', 'ชาบูชิ' : 'shabushi', 'อารามณ์' : 'อารมณ์', 'ส่งน' : 'ส่วน', 'สไลต์' : 'สไลด์', 'ประยุคก์' : 'ประยุกต์',
    'ไอติม' : 'icecream', 'พาราไดส์' : 'paradise', 'พาราไดซ์' : 'paradise', 'อัพเดท' : 'update', 'นาราย' : 'narai ', 'พรีเมี่ยม' : 'พรีเมียม',
    'บ๋วย' : 'บ๊วย', 'เฟรนไชส์' : 'แฟรนไชส์', 'บุพเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ่' : 'บุฟเฟ่ต์', 'บุฟเฟ' : 'บุฟเฟ่ต์', 'บุฟเฟต' : 'บุฟเฟ่ต์', 'รสชาต' : 'รสชาติ', 'รสชาติิ' : 'รสชาติ'
}

# Cleansing
def cleansing_text(text):

    text = text.strip().lower()

    # for old,new in word_correction_dict.items():
    #     text = text.replace(old, new)

    # Remove empty spaces
    text = re.sub(r'\u200b', '', text)
    text = re.sub(r'[\r\n]', '', text)

    # Replace punctuations and emoji
    text = re.sub(r'[🤣😆ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', text)

    # Replace numbers
    text = re.sub(r'\d',' ', text)
    text = re.sub(r'[0-9]','',text)
    text = re.sub(r'[๐-๙]','',text)

    return text

df['clean_text'] = df.apply(lambda x: cleansing_text(x['Headline'] + ' ' + x['Review']), axis=1)
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,clean_text
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก มาทานช่...
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,เนื้อดี ไอติมดี คุ้มค่าดี ถ้านึกถึงชาบูที่มีเน...
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...


In [42]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed = [' ', '  ','    ', '       ','   ','\n', 'ร้าน', 'ที่', 'นะคะ', 'เค้า', 'ผม', 'คน', 'คุ', 'กก', 'ววว', 'เ']
screening = stopwords + removed

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening:
      merged = merged + ',' + word
  return merged[1:]

df['review_tokenized'] = df['clean_text'].apply(lambda x: tokenize_with_space(x))
df.head()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,clean_text,review_tokenized,topics,score
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"บริการ,ดีมาก,พนักงาน,น่ารัก,สะอาดสะอ้าน,ใส่ใจ,...",2,0.998406
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่...,"รสชาติ,เหมือน,สาขา,สาขา,บริการ,ดี,ที่นั่ง,ดี,น...",4,0.997859
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก มาทานช่...,"ชาบู,พรีเมี่ยม,สไตล์,ญี่ปุ่น,เนื้อ,ดีมาก,ทาน,น...",1,0.999294
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,เนื้อดี ไอติมดี คุ้มค่าดี ถ้านึกถึงชาบูที่มีเน...,"เนื้อ,ดี,ไอติม,ดี,คุ้มค่า,ดี,นึกถึง,ชาบู,เนื้อ...",0,0.997409
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...,"อาหาร,คุณภาพ,บริการ,ดี,เดิน,หน้า,ได้กลิ่น,ชาบู...",1,0.997459


In [43]:
documents = df['review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [44]:
print(dictionary.token2id.keys())

dict_keys(['กิน', 'ข้างนอก', 'ความต้องการ', 'คอ', 'คุณภาพ', 'คุณภาพดี', 'คุ้ม', 'จำกัด', 'ซุป', 'ดำ', 'ดี', 'ดีมาก', 'ถาด', 'ถาม', 'น่ารัก', 'น้ำ', 'น้ำจิ้ม', 'บริการ', 'บาท', 'ประทับใจ', 'พนักงาน', 'พอกับ', 'รสชาติ', 'ราคา', 'สรุป', 'สะอาดสะอ้าน', 'สัน', 'หลากหลาย', 'อร่อย', 'อันนี้', 'อาหาร', 'เคลม', 'เซ็ท', 'เต้าเจี้ยว', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แบบนี้', 'แปลกใหม่', 'แฟน', 'โดยรวม', 'โมจิ', 'ใส่ใจ', 'ไม่ต้อง', 'ไอติม', '            ', 'ctw', 'terminal', 'กา', 'การงด', 'คิดถึง', 'ชาบู', 'ญี่ปุ่น', 'ฐาน', 'ทะยอย', 'ทำงาน', 'ที่ทาง', 'ที่นั่ง', 'นาที', 'นึกถึง', 'พนักงานบริการ', 'พระราม', 'พรีเมียม', 'พาราไดซ์', 'มาตรา', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'สู้', 'ออร์เดอร์', 'เครื่องดื่ม', 'เจอ', 'เป็นกันเอง', 'เหมือน', 'เหวี่ยง', 'แจ้ง', 'โม', 'โม่', 'โอเค', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คาว', 'คุ้มค่า', 'จาน', 'จิ้ม', 'จืด', 'ฉาก', 'ชอบ', 'ชาเขียว', 'ชิ้น', 'ซอย', 'ซอส', 'ตก', 'ตะ', 'ตัก', 'ตามน้ำ', 'ต่างกัน', 'ถั่ว', 'ถ่ายรูป', 'ทาน', 'นั่ง', 'นุ่ม', 'บด', 'บาร

In [45]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

**Topic Modeling**

In [55]:
num_topics = 5
chunksize = 4000   # size of the doc looked at every pass
passes = 20         # number of passes through documents
iterations = 100    
eval_every = 1     # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 363 ms, sys: 4.45 ms, total: 367 ms
Wall time: 366 ms


In [56]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.062666  0.012820       1        1  41.187841
2     -0.034944  0.035895       2        1  19.644050
4     -0.010204 -0.000592       3        1  14.300169
3     -0.009400 -0.042304       4        1  14.018292
1     -0.008118 -0.005819       5        1  10.849648, topic_info=         Term       Freq      Total Category  logprob  loglift
18        บาท  14.000000  14.000000  Default  30.0000  30.0000
15        น้ำ  29.000000  29.000000  Default  29.0000  29.0000
23       ราคา  21.000000  21.000000  Default  28.0000  28.0000
209  บุฟเฟ่ต์   8.000000   8.000000  Default  27.0000  27.0000
0         กิน  22.000000  22.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
257        ชิ   0.960598   8.653305   Topic5  -5.0776   0.0229
22     รสชาติ   0.981462  15.030139   Topic5  -5.0561  -0.5077
120        สด   0.949963  10.560334   Topic5  -5.0887  -0.1874
223      เดอะ   0.878278   6.663679   Topic5  -5.1672   0.1946
117       ลอง   0.875135   7.348968   Topic5  -5.1708   0.0931

[346 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
253       1  0.279538         
253       2  0.372718         
253       3  0.186359         
253       4  0.093179         
253       5  0.093179         
...     ...       ...      ...
44        2  0.230401    ไอติม
44        3  0.115201    ไอติม
44        4  0.115201    ไอติม
44        5  0.115201    ไอติม
168       1  0.736916  ไอศกรีม

[382 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 4, 2])

In [62]:
model.show_topic(3)

[('น้ำ', 0.027886277),
 ('ชาบู', 0.022367053),
 ('กิน', 0.020424433),
 ('รสชาติ', 0.018607894),
 ('อาหาร', 0.01844138),
 ('อร่อย', 0.018225567),
 ('ราคา', 0.01781145),
 ('เนื้อ', 0.015819734),
 ('ซุป', 0.014937248),
 ('บริการ', 0.01428571)]

In [58]:
df['topics'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [59]:
df[['Restaurant', 'review_tokenized', 'topics', 'score']]

,Restaurant,review_tokenized,topics,score
0,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"บริการ,ดีมาก,พนักงาน,น่ารัก,สะอาดสะอ้าน,ใส่ใจ,...",3,0.998311
1,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"รสชาติ,เหมือน,สาขา,สาขา,บริการ,ดี,ที่นั่ง,ดี,น...",0,0.998044
2,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"ชาบู,พรีเมี่ยม,สไตล์,ญี่ปุ่น,เนื้อ,ดีมาก,ทาน,น...",0,0.999297
3,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"เนื้อ,ดี,ไอติม,ดี,คุ้มค่า,ดี,นึกถึง,ชาบู,เนื้อ...",0,0.997178
4,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"อาหาร,คุณภาพ,บริการ,ดี,เดิน,หน้า,ได้กลิ่น,ชาบู...",1,0.997086
5,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"กิน,อิ่ม,momo,บุ,ฟเฟ่,ชาบู,แนว,ญี่ปุ่น,สาย,เนื...",0,0.996923
6,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"ชาบู,สุก,กี้,สไลต์,ญี่ปุ่น,ทรู,number, ,โ...",3,0.998237
7,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,"ชาบู,เฟรนไชส์,ราคา,เหมาะสม,คุณภาพ,คุ้ม,น้ำ,ซุป...",3,0.998003
8,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,"อร่อย,คุ้ม,ทาน,เล่น,าน,ขา,บู,ชิ,ตอน,หิว,ไม่งั้...",2,0.997140
9,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,"ทาน,ชอบ,กุ้ง,ทอด,เท,ม,ปุระ,ชอบ,กุ้ง,ทอด,เท,ม,ป...",0,0.998359
